In [1]:
# merge file 
import os
import pandas as pd
import numpy as np

# =========================
# CẤU HÌNH (LOCAL)
# =========================

# ROOT dataset trên máy bạn
DATA_ROOT = r"C:\Users\haiye\OneDrive\Desktop\Nam3_Ki1\Machine_learning\BTL\mallorn-astronomical-classification-challenge"

# Thư mục output (tạo cùng thư mục notebook)
OUT_ROOT = "cleaned_data"

SPLITS = range(1, 21)

REQUIRED_COLS = ["object_id", "Time (MJD)", "Flux", "Flux_err", "Filter"]
VALID_FILTERS = {"u", "g", "r", "i", "z", "y"}

os.makedirs(OUT_ROOT, exist_ok=True)
train_log = pd.read_csv("train_log.csv")
test_log = pd.read_csv("test_log.csv")
# =========================
# HÀM CLEAN
# =========================
def clean_lightcurve_file(csv_path, is_test_set=False):
    if not os.path.exists(csv_path):
        print(f"!!! Warning: Không tìm thấy file {csv_path}")
        return pd.DataFrame(), 0, 0

    lc = pd.read_csv(csv_path)
    n_before = len(lc)

    original_ids = lc['object_id'].unique() if is_test_set else None

    # ---- 1. Lọc cơ bản ----
    mask = (
        lc[REQUIRED_COLS].notna().all(axis=1) &
        (lc["Flux_err"] > 0) &
        (lc["Flux_err"] < 1e6) &
        (lc["Filter"].isin(VALID_FILTERS)) &
        (lc["Flux"].abs() <= 1e10)
    )

    lc_clean = lc[mask].copy()

    # ---- 2. GỘP TRÙNG (Weighted Average) ----
    lc_clean['w'] = 1.0 / (lc_clean['Flux_err'] ** 2)
    lc_clean['flux_w'] = lc_clean['Flux'] * lc_clean['w']

    group_cols = ["object_id", "Time (MJD)", "Filter"]

    lc_grouped = (
        lc_clean
        .groupby(group_cols)[['flux_w', 'w']]
        .sum()
        .reset_index()
    )

    lc_grouped['Flux'] = lc_grouped['flux_w'] / lc_grouped['w']
    lc_grouped['Flux_err'] = 1.0 / np.sqrt(lc_grouped['w'])

    lc_final = lc_grouped[REQUIRED_COLS].copy()

    # ---- 3. CỨU HỘ TEST ----
    if is_test_set:
        remaining_ids = set(lc_final['object_id'].unique())
        missing_ids = set(original_ids) - remaining_ids

        if missing_ids:
            print(f"   ⚠️ CỨU {len(missing_ids)} ID bị mất trong TEST")
            rescue_df = lc[lc['object_id'].isin(missing_ids)].copy()
            rescue_df['Flux'] = rescue_df['Flux'].fillna(0)
            bad_err_mask = (rescue_df['Flux_err'].isna()) | (rescue_df['Flux_err'] <= 0)
            rescue_df.loc[bad_err_mask, 'Flux_err'] = 100.0
            
            rescue_df = rescue_df[REQUIRED_COLS]

            lc_final = pd.concat([lc_final, rescue_df], ignore_index=True)

    # ---- 4. Sort ----
    lc_final = lc_final.sort_values(["object_id", "Time (MJD)"])

    n_after = len(lc_final)


    return lc_final, n_before, n_after



In [2]:
from extinction import fitzpatrick99

def apply_extinction_correction(lc_df, metadata_df):
    print("--- BẮT ĐẦU GIAI ĐOẠN 2: EXTINCTION CORRECTION ---")
    
    # 1. Định nghĩa bước sóng (như cũ)
    WAVELENGTHS = {
        "u": 3641.0, "g": 4704.0, "r": 6155.0,
        "i": 7504.0, "z": 8695.0, "y": 10056.0,
    }
    
    # 2. Merge Metadata
    meta = metadata_df.copy()
    
    # Kiểm tra xem cột trong metadata là 'EBV' hay 'mwebv'
    ebv_col = 'EBV' 
    if ebv_col not in meta.columns:
        raise ValueError("Metadata thiếu cột 'EBV'")

    # Merge (chú ý chỉ lấy đúng cột cần)
    lc_final = lc_df.merge(meta[['object_id', ebv_col]], on='object_id', how='left')
    
    # Fillna cho EBV nếu bị thiếu (mặc định là 0 - không sửa)
    if lc_final[ebv_col].isna().any():
        print(f"Cảnh báo: Có {lc_final[ebv_col].isna().sum()} dòng thiếu EBV. Điền 0.")
        lc_final[ebv_col] = lc_final[ebv_col].fillna(0)

    # 3. Tính toán Extinction (Vectorized - Không vòng lặp av)
    # Lấy giá trị EBV và clip cho an toàn
    ebv_values = np.clip(lc_final[ebv_col].values, 0, 2.0) # Clip 1.0 hơi gắt, 10.0 an toàn hơn cho vùng tối
    high_ebv = (lc_final[ebv_col] > 2.0).sum()
    if high_ebv > 0:
        print(f"   ⚠️ Warning: {high_ebv} points have EBV > 2.0 (clipped)")
        
    # Tạo mảng A_lambda chứa giá trị dập tắt cho từng dòng
    A_lambda = np.zeros(len(lc_final), dtype=float)
    
    # Tính hệ số cho từng filter (Nhanh hơn loop theo EBV)
    # R_v = 3.1 (Milky Way average)
    for filt, wave in WAVELENGTHS.items():
        mask = (lc_final['Filter'] == filt)
        if not mask.any(): continue
        
        # Tính A_lambda cho EBV=1.0 (Unit Extinction)
        # fitzpatrick99(wave, a_v, r_v) -> a_v ở đây ta để 3.1 vì A_V = R_V * E(B-V)
        # Khi E(B-V) = 1 thì A_V = 3.1
        unit_ext = fitzpatrick99(np.array([wave]), 3.1, 3.1)[0]
        
        # Extinction thực tế = Unit_Ext * EBV thực tế
        A_lambda[mask] = unit_ext * ebv_values[mask]

    # 4. Áp dụng công thức hiệu chỉnh Flux
    # Công thức: F_true = F_obs * 10^(0.4 * A_lambda)
    # Bạn dùng 10**(A/2.5) chính là 10**(0.4*A) -> ĐÚNG
    corr_factor = 10.0 ** (0.4 * A_lambda)
    
    lc_final['Flux'] = lc_final['Flux'] * corr_factor
    lc_final['Flux_err'] = lc_final['Flux_err'] * corr_factor
    
    # 5. Dọn dẹp (Sửa lỗi inplace ở đây)
    lc_final = lc_final.drop(columns=[ebv_col]) 
    
    print(f"--- HOÀN THÀNH (Đã sửa {len(lc_final)} điểm dữ liệu) ---")
    return lc_final

In [ ]:
def apply_quality_filters(lc_df):
    """
    Filter objects that are too poor for feature engineering
    """
    
    # 1. Calculate SNR
    lc_df['SNR'] = lc_df['Flux'] / lc_df['Flux_err']
    
    # 2. Per-object statistics
    obj_stats = lc_df.groupby('object_id').agg({
        'object_id': 'size',  # n_obs
        'SNR': lambda x: (x > 3).sum(),  # n_detections
        'Time (MJD)': lambda x: x.max() - x.min(),  # time_span
        'Filter': 'nunique'  # n_filters
    }).rename(columns={
        'object_id': 'n_obs',
        'SNR': 'n_det',
        'Time (MJD)': 'time_span',
        'Filter': 'n_filters'
    })
    
    # 3. Apply filters (TRAIN only - keep all TEST)
    valid_objects = obj_stats[
        (obj_stats['n_obs'] >= 5) &      # Ít nhất 5 observations
        (obj_stats['n_det'] >= 3) &      # Ít nhất 3 detections
        (obj_stats['time_span'] >= 3) &  # Ít nhất 3 ngày coverage
        (obj_stats['n_filters'] >= 1)    # Ít nhất 1 filter (loose)
    ].index
    
    print(f"   Quality filter: {len(valid_objects)}/{len(obj_stats)} objects pass")
    lc_df = lc_df.drop(columns=['SNR'])
    
    return lc_df[lc_df['object_id'].isin(valid_objects)]

In [4]:
PATH_TO_TRAIN_LOG = ("train_log.csv") 
PATH_TO_TEST_LOG = ("test_log.csv")

print("===== 1. LOADING METADATA (LOG FILES) =====")
try:
    meta_train_master = pd.read_csv(PATH_TO_TRAIN_LOG)
    print(f"✅ Loaded Train Log: {len(meta_train_master)} objects")
    
    meta_test_master = pd.read_csv(PATH_TO_TEST_LOG)
    print(f"✅ Loaded Test Log: {len(meta_test_master)} objects")
except FileNotFoundError as e:
    print(f"❌ Error: Không tìm thấy file log. Kiểm tra lại đường dẫn!\n{e}")
    # Dừng chương trình nếu không có metadata (vì không khử bụi được)
    raise e

===== 1. LOADING METADATA (LOG FILES) =====
✅ Loaded Train Log: 3043 objects
✅ Loaded Test Log: 7135 objects


In [ ]:
print("\n===== 2. START PROCESSING SPLITS =====")

if not os.path.exists(OUT_ROOT):
    os.makedirs(OUT_ROOT)

for i in SPLITS:
    split_name = f"Split_{i:02d}"
    split_dir = os.path.join(DATA_ROOT, split_name)

    if not os.path.exists(split_dir):
        split_name = f"split_{i:02d}"
        split_dir = os.path.join(DATA_ROOT, split_name)
    
    if not os.path.exists(split_dir):
        continue # Bỏ qua nếu không tìm thấy folder

    print(f"\nProcessing {split_name}...")

    for mode in ["train", "test"]:
        lc_path = os.path.join(split_dir, f"{mode}_full_lightcurves.csv")
        
        if not os.path.exists(lc_path):
            continue

        # 1. Clean Basic
        lc_step1, n_raw, _ = clean_lightcurve_file(lc_path, is_test_set=(mode == "test"))
        
        if lc_step1.empty:
            continue

        # 2. Extinction Correction (Chọn Metadata đúng)
        current_meta = meta_train_master if mode == "train" else meta_test_master
        
        # Gọi hàm khử bụi
        lc_step2 = apply_extinction_correction(lc_step1, current_meta)

        if mode == "train":
            # Áp dụng quality filter cho train set
            lc_step2 = apply_quality_filters(lc_step2)

        lc_final = lc_step2.copy()

        # Lưu file
        out_name = f"split_{i:02d}_{mode}_processed.csv"
        out_path = os.path.join(OUT_ROOT, out_name)
        lc_final.to_csv(out_path, index=False)

        n_final = len(lc_final)
        print(f"   | {mode.upper():5s} | Raw: {n_raw:7d} -> Final: {n_final:7d} | Saved.")

print("\n===== ALL DONE =====")


===== 2. START PROCESSING SPLITS =====

Processing Split_01...
--- BẮT ĐẦU GIAI ĐOẠN 2: EXTINCTION CORRECTION ---
--- HOÀN THÀNH (Đã sửa 26313 điểm dữ liệu) ---
   Quality filter: 155/155 objects pass
   | TRAIN | Raw:   26324 -> Final:   26313 | Saved.
--- BẮT ĐẦU GIAI ĐOẠN 2: EXTINCTION CORRECTION ---
--- HOÀN THÀNH (Đã sửa 59212 điểm dữ liệu) ---
   | TEST  | Raw:   59235 -> Final:   59212 | Saved.

Processing Split_02...
--- BẮT ĐẦU GIAI ĐOẠN 2: EXTINCTION CORRECTION ---
--- HOÀN THÀNH (Đã sửa 25603 điểm dữ liệu) ---
   Quality filter: 170/170 objects pass
   | TRAIN | Raw:   25609 -> Final:   25603 | Saved.
--- BẮT ĐẦU GIAI ĐOẠN 2: EXTINCTION CORRECTION ---
--- HOÀN THÀNH (Đã sửa 71221 điểm dữ liệu) ---
   | TEST  | Raw:   71229 -> Final:   71221 | Saved.

Processing Split_03...
--- BẮT ĐẦU GIAI ĐOẠN 2: EXTINCTION CORRECTION ---
--- HOÀN THÀNH (Đã sửa 21671 điểm dữ liệu) ---
   Quality filter: 138/138 objects pass
   | TRAIN | Raw:   21676 -> Final:   21671 | Saved.
--- BẮT ĐẦU G